<a href="https://colab.research.google.com/github/kahuroA/SQL-PRACTICE/blob/main/SQL_MINI_PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#mounting collab on drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#check working directory
!pwd

/content


In [4]:
#changing to working directory
root_dir = '/content/drive/MyDrive'
project_folder = '/content/drive/MyDrive/SQL-DATA'

In [5]:
#importation of libraries
import pandas as pd
import os
os.chdir(root_dir)
os.chdir(project_folder)

In [6]:
!pwd

/content/drive/MyDrive/SQL-DATA


**SQL MINI-PROJECT** *

In [7]:
#load the sql extension
%load_ext sql
#create a temporary connection of sqlite
%sql sqlite://

'Connected: @None'

In [8]:
#open the data files using python, read with pandas and store in sql table
with open('GrandElectors_by_state.csv','r') as f:
  Grand_Electors=pd.read_csv(f, index_col=0, encoding='utf-8')
%sql DROP TABLE IF EXISTS Grand_Electors;
%sql PERSIST Grand_Electors;
%sql SELECT * FROM Grand_Electors LIMIT 5;

 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.


State,GrandElectors
Alabama,9
Alaska,3
Arizona,11
Arkansas,6
California,55


In [9]:
#open the population csv with pandas and store and read the dataframe using sql table
with open('Population_by_state.csv','r') as f:
  Population=pd.read_csv(f, index_col=0, encoding='utf-8')
%sql DROP TABLE IF EXISTS Population;
%sql PERSIST Population;
%sql SELECT * FROM Population LIMIT 5;

 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.


State,Population
CALIFORNIA,39144818
TEXAS,27469114
FLORIDA,20271272
NEW YORK,19795791
ILLINOIS,12859995


In [ ]:
# capitalize the states in the Grand_Electors table so as to enable an inner join
%sql UPDATE Grand_Electors SET State=UPPER(State);

 * sqlite://
51 rows affected.


[]

In [ ]:
%%sql
--Make the inner join between Grand_Electors and Population, store the result in a new table called Combined
CREATE TABLE Combined AS
SELECT * FROM Grand_Electors
INNER JOIN Population  on Population.State = Grand_Electors.State;

 * sqlite://
Done.


[]

In [ ]:
%%sql
--view the combined table
SELECT * FROM Combined LIMIT 5;

 * sqlite://
Done.


State,GrandElectors,State:1,Population
ALABAMA,9,ALABAMA,4858979
ALASKA,3,ALASKA,738432
ARIZONA,11,ARIZONA,6828065
ARKANSAS,6,ARKANSAS,2978204
CALIFORNIA,55,CALIFORNIA,39144818


In [ ]:
%%sql
--We will drop the redundant column State:1 by creating a table Combined_new with relevant columns from Combined.
CREATE TABLE Combined_new (
  State  TEXT PRIMARY KEY,
  GrandElectors  int,
  Population  int
);
--After creating the table with relevant columns from Combine table, we will insert columns from Combined table 
INSERT INTO Combined_new SELECT State, GrandElectors, Population FROM Combined;
--We will then drop the table Combined
DROP TABLE Combined;
--finally we will alter the new table's name and give it the name Combined
ALTER TABLE Combined_new RENAME TO Combined;

 * sqlite://
Done.
Done.
Done.
Done.


[]

In [ ]:
%%sql
--view the table combined
SELECT * FROM Combined LIMIT 5;

 * sqlite://
Done.


State,GrandElectors,Population
ALABAMA,9,4858979
ALASKA,3,738432
ARIZONA,11,6828065
ARKANSAS,6,2978204
CALIFORNIA,55,39144818


2. Your boss wants you to change the name of the "District of Columbia" state to its short version "DC". Please do that.

In [ ]:
%%sql
--We update the state column with a condition of where we need to do the update
UPDATE Combined
SET State='DC'
WHERE State='DISTRICT OF COLUMBIA'

 * sqlite://
Done.


[]

3. To compute the ratio between the number of grand electors and the population. Please create a new column with that ratio.

In [ ]:
%%sql
--create a new table with the ratio column
CREATE TABLE Combined_new AS
SELECT *, CAST(Combined.GrandElectors AS FLOAT)/CAST(Combined.Population AS FLOAT) AS Ratio
FROM   Combined;
--drop the old table Combined
DROP TABLE Combined;
--Rename the new table with ratio column as Combined
ALTER TABLE Combined_new Rename to Combined;

 * sqlite://
Done.
Done.
Done.


[]

In [ ]:
%%sql
--Let us view the table with the new ratio column
SELECT * FROM Combined LIMIT 5;

 * sqlite://
Done.


State,GrandElectors,Population,Ratio
ALABAMA,9,4858979,1.8522409749043987e-06
ALASKA,3,738432,4.0626625065002604e-06
ARIZONA,11,6828065,1.610998137832607e-06
ARKANSAS,6,2978204,2.0146370094190997e-06
CALIFORNIA,55,39144818,1.4050391037710278e-06


4. To order the states by decreasing ratio of Grand Electors per capita. That will make our priority list

In [ ]:
%%sql
CREATE TABLE Sorted AS
SELECT * FROM Combined
ORDER BY Ratio DESC;

 * sqlite://
Done.


[]

In [ ]:
%%sql
--let us view the new table with ratio sorted in decreasing order
SELECT * FROM Sorted LIMIT 5;

 * sqlite://
Done.


State,GrandElectors,Population,Ratio
WYOMING,3,586107,5.118519314732634e-06
VERMONT,3,626042,4.792010759661493e-06
DC,3,672228,4.462771559649405e-06
ALASKA,3,738432,4.0626625065002604e-06
NORTH DAKOTA,3,756927,3.963394092164766e-06


5. To compute the running total of Grand Electors in that sorted list

In [ ]:
%%sql
CREATE TABLE Sorted_Cummulative AS
SELECT
    t.State,t.GrandElectors,t.Population,t.Ratio,
    (SELECT Sum(r.GrandElectors) FROM Sorted as r WHERE t.Ratio <= r.Ratio) AS Cummulative
FROM Sorted as t
Group By t.Ratio ORDER BY t.Ratio DESC;

 * sqlite://
Done.


[]

In [ ]:
%%sql
--Let us view the first 5 columns of the new table with the cummulative sum.
SELECT * FROM Sorted_Cummulative;

 * sqlite://
Done.


State,GrandElectors,Population,Ratio,Cummulative
WYOMING,3,586107,5.118519314732634e-06,3
VERMONT,3,626042,4.792010759661493e-06,6
DC,3,672228,4.462771559649405e-06,9
ALASKA,3,738432,4.0626625065002604e-06,12
NORTH DAKOTA,3,756927,3.963394092164766e-06,15
RHODE ISLAND,4,1056298,3.7868101615263876e-06,19
SOUTH DAKOTA,3,858469,3.4945932817609025e-06,22
DELAWARE,3,945934,3.1714686225466047e-06,25
MAINE,4,1329328,3.009039153617467e-06,29
NEW HAMPSHIRE,4,1330608,3.0061445594795764e-06,33


6. Independently, to compute the half of the total of Grand Electors overall (in the whole country)

In [ ]:
%%sql
SELECT SUM(GrandElectors)/2 FROM Sorted_Cummulative;

 * sqlite://
Done.


SUM(GrandElectors)/2
269


In [ ]:
%%sql
SELECT State,GrandElectors FROM Sorted_Cummulative WHERE GrandElectors= (SELECT MAX(GrandElectors) FROM Sorted_Cummulative);

 * sqlite://
Done.


State,GrandElectors
CALIFORNIA,55


In [ ]:
%%sql
SELECT State,GrandElectors FROM Sorted_Cummulative WHERE GrandElectors= (SELECT MIN(GrandElectors) FROM Sorted_Cummulative);

 * sqlite://
Done.


State,GrandElectors
WYOMING,3
VERMONT,3
DC,3
ALASKA,3
NORTH DAKOTA,3
SOUTH DAKOTA,3
DELAWARE,3
MONTANA,3


In [ ]:
%%sql
SELECT State,Population FROM Sorted_Cummulative WHERE Population= (SELECT MIN(Population) FROM Sorted_Cummulative);

 * sqlite://
Done.


State,Population
WYOMING,586107


In [ ]:
%%sql
SELECT State,Population FROM Sorted_Cummulative WHERE Population= (SELECT MAX(Population) FROM Sorted_Cummulative);

 * sqlite://
Done.


State,Population
CALIFORNIA,39144818


In [ ]:
%%sql
SELECT AVG(GrandElectors) FROM Sorted_Cummulative;

 * sqlite://
Done.


AVG(GrandElectors)
10.549019607843137


In [ ]:
%%sql
SELECT AVG(Population) FROM Sorted_Cummulative;

 * sqlite://
Done.


AVG(Population)
6302329.803921568


In [ ]:
%%sql
SELECT State, Population FROM Sorted_Cummulative
WHERE Population>=(SELECT AVG(Population) FROM Sorted_Cummulative);

 * sqlite://
Done.


State,Population
WASHINGTON,7170351
TENNESSEE,6600299
INDIANA,6619680
MASSACHUSETTS,6794422
MICHIGAN,9922576
ARIZONA,6828065
GEORGIA,10214860
NEW JERSEY,8958013
PENNSYLVANIA,12802503
ILLINOIS,12859995


In [ ]:
%%sql
SELECT GrandElectors
FROM Sorted_Cummulative
ORDER BY GrandElectors
LIMIT 1
OFFSET (SELECT COUNT(*)
        FROM Sorted_Cummulative) / 2

 * sqlite://
Done.


GrandElectors
8


Calculate the frequency distribution of Grand Electors

In [ ]:
%%sql
SELECT GrandElectors,COUNT(GrandElectors) AS Frequency
FROM Sorted_Cummulative
GROUP BY GrandElectors
ORDER BY Frequency Desc;

 * sqlite://
Done.


GrandElectors,Frequency
3,8
6,6
4,5
10,4
11,4
5,3
7,3
9,3
8,2
16,2


7. To filter our sorted list of states in order to keep only the (top) ones enabling us to reach the computed threshold. (the other states can be ignored). That is our target list.

In [ ]:
%%sql
CREATE TABLE Target_List AS
SELECT * FROM Sorted_Cummulative WHERE Cummulative<=269;

 * sqlite://
Done.


[]

In [ ]:
%%sql
--Let us view our target list
SELECT * FROM Target_List

 * sqlite://
Done.


State,GrandElectors,Population,Ratio,Cummulative
WYOMING,3,586107,5.118519314732634e-06,3
VERMONT,3,626042,4.792010759661493e-06,6
DC,3,672228,4.462771559649405e-06,9
ALASKA,3,738432,4.0626625065002604e-06,12
NORTH DAKOTA,3,756927,3.963394092164766e-06,15
RHODE ISLAND,4,1056298,3.7868101615263876e-06,19
SOUTH DAKOTA,3,858469,3.4945932817609025e-06,22
DELAWARE,3,945934,3.1714686225466047e-06,25
MAINE,4,1329328,3.009039153617467e-06,29
NEW HAMPSHIRE,4,1330608,3.0061445594795764e-06,33


Add the first state for which the running total is larger than the threshold.

In [ ]:
%%sql
--Let us add the first state for which the running total is larger than the threshold
INSERT INTO Target_List(State, GrandElectors, Population, Ratio, Cummulative)
SELECT * FROM Sorted_Cummulative WHERE Cummulative>269 LIMIT 1;

 * sqlite://
Done.


[]

In [ ]:
%%sql
--Let us view the Target List with the additional row
SELECT * FROM Target_List

 * sqlite://
Done.


State,GrandElectors,Population,Ratio,Cummulative
WYOMING,3,586107,5.118519314732634e-06,3
VERMONT,3,626042,4.792010759661493e-06,6
DC,3,672228,4.462771559649405e-06,9
ALASKA,3,738432,4.0626625065002604e-06,12
NORTH DAKOTA,3,756927,3.963394092164766e-06,15
RHODE ISLAND,4,1056298,3.7868101615263876e-06,19
SOUTH DAKOTA,3,858469,3.4945932817609025e-06,22
DELAWARE,3,945934,3.1714686225466047e-06,25
MAINE,4,1329328,3.009039153617467e-06,29
NEW HAMPSHIRE,4,1330608,3.0061445594795764e-06,33


How many states do you end up within the target list?

In [ ]:
%%sql
SELECT COUNT(State) FROM Target_List;

 * sqlite://
Done.


COUNT(State)
40


Is it a small or a large number? 

In [ ]:
(40/51)*100

78.43137254901961

The total number of states in our target list is 78.4% of the total states in the US. So this is a large number.

 Do you think it would be a good recommendation to target those states?

In [ ]:
%%sql
SELECT SUM(GrandElectors) FROM Target_List;

 * sqlite://
Done.


SUM(GrandElectors)
271
